<a href="https://colab.research.google.com/github/team05-MLOps-pipeline/kafka-consumer-db/blob/main/kafka_consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install confluent-kafka mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [2]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00


In [10]:
# 24시간이 넘는 수를 처리해주는 코드
# 종료코드 적용 전

from confluent_kafka import Consumer, KafkaException
import pymysql
import datetime
import json

# Kafka consumer 설정
conf = {'bootstrap.servers': 'ip:9094',
        'group.id': 'mygroup',
        'auto.offset.reset': 'earliest'}
consumer = Consumer(conf)

# MySQL 연결 설정
conn = pymysql.connect(host='ip',
                       user='eun',
                       port=13306,
                       password='12341234!',
                       db='eun_test',
                       charset='utf8')
cursor = conn.cursor()

def convert_date(kr_date_str):
    # "2023.10.13. 오후 1:09" --> "2023-10-13 13:09"
    date_str, am_pm, time_str = kr_date_str.split()
    hour, minute = map(int, time_str.split(':'))

    if am_pm == "오후" and hour < 12:
        hour += 12
    elif am_pm == '오전' and hour == 12:
        hour = 0

    return f"{date_str.replace('.', '-')} {hour:02d}:{minute:02d}:00"

try:
    # Kafka topic 구독
    consumer.subscribe(['euntest'])

    while True:
        msg = consumer.poll(1.0)  # timeout is set to 1 second

        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            value = msg.value().decode('utf-8')
            print('Received message: {}'.format(value))

            # MySQL에 데이터 적재하기
            sql = "INSERT INTO Article (category_id, news_url, company_name, title, author_info, create_date,\
                   modify_date,content,image_url) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

            data=json.loads(value)

            create_date_converted=convert_date(data['create_date'])

            # try:
            #     cursor.execute(sql,
            #                    (data['category'], data['url'], data['site_name'], data['title'],
            #                     data['author_info'], create_date_converted , None,
            #                     data['content'], data['image_url']))
            #     conn.commit()

            # except pymysql.err.IntegrityError as e:
            #     if e.args[0] == 1062:
            #         print(f"Duplicate entry {data['url']} skipped.")
            #     else:
            #         raise e
            cursor.execute(sql,
                           (data['category'], data['url'], data['site_name'], data['title'],
                            data['author_info'], create_date_converted , None,
                            data['content'], data['image_url']))

            conn.commit()  # 변경 사항 커밋

except KeyboardInterrupt:
    pass

finally:

   consumer.close()
   cursor.close()
   conn.close()